# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon) in themoving domain case (the upper boundary will move).  The unkwons are the velocity $u$, the elevation head $h$ and the height of the upper boundary $\eta$.

Let $\Omega=(0,1)\times(0,\eta)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$K$ the matrix permeability, we want to solve the following problem: find $(\bm{u}, h)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1} {\bm{u}} + \nabla h = {0}\\
S_s \frac{\partial{h}}{\partial t} + \nabla \cdot {u} = f
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$

In order to solve the problem, we will perfom a change of coordinates to a reference domain $\hat{\Omega}=(0,1)^2$ through the (linear) trasnformation $R : \Omega \rightarrow \hat{\Omega}$ (and its inverse function $D : \hat{\Omega} \rightarrow \Omega$).
Recall that $\hat{\nabla}R=(\nabla D)^{-1}$.

Let $\hat{h}$ and $\hat{\bm{u}}$ be $h$ and $\bm{u}$ respectevely in the reference domain and let $\hat{K}$ be the transformed permeability matrix, defined as $\hat{K}=det(\hat{\nabla}D) (\hat{\nabla} D)^{-1} K (\hat{\nabla} D)^{-T}$.

The equation describing the motion of $\partial_{top}\Omega$ is:
$$

\phi \frac{\partial \eta}{\partial t} = \hat{u_3}

$$

The transformed equations in $\hat{\Omega}$ is:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\hat{K}(\hat{h})^{-1} {\hat{u}} + \hat{\nabla} \hat{h} = {0}\\
\hat{S}_s \frac{\partial{\hat{h}}}{\partial t} + \hat{\nabla} \cdot {\hat{\bm{u}}} = f
\end{array}
&\text{in } \hat{\Omega}
\end{array}
\right.
$$
with boundary conditions:
$$ \hat{h} = \eta \text{ on } \Gamma \qquad \hat{h} = \ell \text{ on } \Gamma_D \qquad \hat{\bm{\nu}} \cdot \hat{\bm{u}} = 0 \text{ on } \Gamma_N$$

The weak formulation will be:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h})^{-1} {\bm{\hat{u}}} \cdot \bm{v} \, d\Omega - \int_{\Omega} h \hat{\nabla} \cdot {\hat{\bm{v}}} \, d\Omega = - \int_{\Gamma_D} h \bm{v} \cdot \bm{\nu} \, d\Omega - \int_{\Gamma} \eta \bm{v} \cdot \bm{\nu} \, d\Omega\\
\int_{\Omega} \hat{S}_s \frac{\partial{\hat{h}}}{\partial t} v \, d\Omega + \int_{\Omega} \hat{\nabla} \cdot {\hat{\bm{u}}} v \, d\Omega = \int_{\Omega} fv \, d\Omega\\
\int_{\Gamma} \phi \frac{\partial \eta}{\partial t} v \, d\sigma = \int_{\Gamma} \hat{u_3} v \, d\sigma
\end{array}
\end{array}
\right.
$$

For the time discretization, we will employ a backward Euler scheme:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h}^{n+1})^{-1} {\bm{\hat{u}}^{n+1}} \cdot \bm{v} \, d\Omega - \int_{\Omega} h^{n+1} \hat{\nabla} \cdot {\hat{\bm{v}}} \, d\Omega = - \int_{\Gamma_D} h^{n+1} \bm{v} \cdot \bm{\nu} \, d\Omega - \int_{\Gamma} \eta^{n+1} \bm{v} \cdot \bm{\nu} \, d\Omega\\
\int_{\Omega} \hat{S}_s^{n+1} \frac{\hat{h}^{n+1} - \hat{h}^{n}}{\Delta t} v \, d\Omega + \int_{\Omega} \hat{\nabla} \cdot {\hat{\bm{u}}^{n+1}} v \, d\Omega = \int_{\Omega} f^{n+1}v \, d\Omega\\
\int_{\Gamma} \phi \eta^{n+1} v \, d\sigma = \Delta t \int_{\Gamma} \hat{\bm{u}}^{n+1} \cdot \bm{\nu} v \, d\sigma + \int_{\Gamma} \phi \eta^{n} v \, d\sigma
\end{array}
\end{array}
\right.
$$

To deal with the non-linear term, we will employ a simple Picard scheme:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h}^{n+1}_k)^{-1} {\bm{\hat{u}_{k+1}}^{n+1}} \cdot \bm{v} \, d\Omega - \int_{\Omega} h^{n+1}_{k+1} \hat{\nabla} \cdot {\hat{\bm{v}}} \, d\Omega = - \int_{\Gamma_D} h^{n+1}_{k+1} \bm{v} \cdot \bm{\nu} \, d\Omega - \int_{\Gamma} \eta^{n+1}_{k+1} \bm{v} \cdot \bm{\nu} \, d\Omega\\
\int_{\Omega} \hat{S}_s^{n+1} \frac{\hat{h}^{n+1}_{k+1} - \hat{h}^{n}}{\Delta t} v \, d\Omega + \int_{\Omega} \hat{\nabla} \cdot {\hat{\bm{u}}^{n+1}_{k+1}} v \, d\Omega = \int_{\Omega} f^{n+1}v \, d\Omega\\
\int_{\Gamma} \phi \eta^{n+1}_{k+1} v \, d\sigma = \Delta t \int_{\Gamma} \hat{\bm{u}}^{n+1}_{k+1} \cdot \bm{\nu} v \, d\sigma + \int_{\Gamma} \phi \eta^{n} v \, d\sigma
\end{array}
\end{array}
\right.
$$

The matrix formulation will be:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
M_u(\bm{h}^{n+1}_{k+1}) \bm{u}^{n+1}_{k+1} + B\bm{h}^{n+1}_{k+1} + B_{\Gamma} \bm{\eta}^{n+1}_{k+1}= \bm{BC}^{n+1}\\
- \Delta t B^T \hat{\bm{u}}^{n+1}_{k+1} + S_s M_{h} \bm{\hat{h}^{n+1}_{k+1}} = \Delta t \bm{F}^{n+1} + M_{h} \bm{\hat{h}^{n}}\\
- \Delta t B_{\Gamma}^T \hat{\bm{u}}^{n+1}_{k+1} + \phi M_{\Gamma} \bm{\eta^{n+1}_{k+1}} = \phi M_{\Gamma} \bm{\eta^{n}}
\end{array}
\end{array}
\right.
$$

$$
\left(
\begin{array}{cc} 
M_u(\bm{h^{n+1}_k}) & B & B_{\Gamma}\\
-\Delta t B^T & S_s M_h & 0\\
-\Delta t B^T_{\Gamma} & 0 & \phi M_{\Gamma}
\end{array}
\right)
\left(
\begin{array}{c} 
\bm{u^{n+1}_{k+1}}\\ 
\bm{h^{n+1}_{k+1}}\\
\bm{\eta^{n+1}_{k+1}}
\end{array}
\right)
=\left(
\begin{array}{c} 
\bm{BC}^{n+1}\\ 
\Delta t \bm{F}^{n+1} + M_h \bm{h}^n\\
\phi M_{\Gamma} \bm{\eta}^n
\end{array}
\right)
$$

We will start to test the method in the case $M_u(\bm{h_k}^{n+1})=\bm{I}$

In [1]:
import numpy as np
import scipy.sparse as sps
import os 
import shutil

import porepy as pp
import pygeon as pg

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
output_directory = 'output'

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [3]:
N = 10
sd = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([sd])

dt = 0.1
T = 10
S_s = 0.3
phi = 0.1

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [4]:
def initial_h_func(x): 
    return (1-x[1])

In [5]:
key = "flow"
bc_val = []
bc_ess = []
initial_pressure = []

velocity_discretization_field = pg.RT0(key)
head_discretization_field = pg.PwConstants(key)

sd, data = mdg.subdomains(return_data=True)[0]

# permeability tensor
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
parameters = {
    "second_order_tensor": perm,
}

pp.initialize_data(sd, data, key, parameters)
    
# with the following steps we identify the portions of the boundary
# to impose the boundary conditions
left_right = np.logical_or(sd.face_centers[0, :] == 0,  sd.face_centers[0, :] == 1)

bottom = sd.face_centers[1, :] == 0
top    = sd.face_centers[1, :] == 1

ess_p_dofs = np.zeros(head_discretization_field.ndof(sd), dtype=bool)

def h_bc(x): return 1

bc_val.append(-velocity_discretization_field.assemble_nat_bc(sd, h_bc, bottom))

bc_ess.append(np.hstack((left_right, ess_p_dofs, np.zeros(N, dtype=bool))))

In [6]:
def assemble_gamma_matrices():
    B_rows = []

    faces = np.where( top == True )[0]

    for i in range(N):
        tmp_vect = np.zeros_like(top, dtype=bool)
        tmp_vect[ faces[i] ] = True
        
        B_rows.append(velocity_discretization_field.assemble_nat_bc(sd, h_bc, tmp_vect).T)
    
    return np.vstack(B_rows), np.eye(N=N)

In [7]:
B_gamma, M_gamma = assemble_gamma_matrices()

$$
\left(
\begin{array}{cc} 
M_u & B^T & B_{\Gamma}^T\\
-\Delta t B & S_s M_h & 0\\
-\Delta t B_{\Gamma} & 0 & \phi M_{\Gamma}
\end{array}
\right)
\left(
\begin{array}{c} 
\bm{u^{n+1}}\\ 
\bm{h^{n+1}}\\
\bm{\eta^{n+1}}
\end{array}
\right)
=\left(
\begin{array}{c} 
\bm{BC}^{n+1}\\ 
M_h \bm{h}^n\\
\phi M_{\Gamma} \bm{\eta}^n
\end{array}
\right)
$$

In [8]:
# construct the local matrices
mass = pg.face_mass(mdg)
div = - pg.cell_mass(mdg, head_discretization_field) * pg.div(mdg)

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape
dof_eta = B_gamma.shape[0]

mass.shape, div.shape, B_gamma.shape

((320, 320), (200, 320), (10, 320))

In [9]:
# assemble the right-hand side
fixex_rhs = np.zeros(dof_p + dof_q + dof_eta)
fixex_rhs[:dof_q] += np.hstack(bc_val)

In [10]:
# construct the local matrices
mass = pg.face_mass(mdg)
div = pg.cell_mass(mdg, head_discretization_field) * pg.div(mdg)

# assemble the saddle point problem
spp = sps.bmat([[         mass,                                                    div.T,     B_gamma.T], 
                [    -dt * div, S_s * head_discretization_field.assemble_mass_matrix(sd),          None],
                [-dt * B_gamma,                                                     None, phi * M_gamma]], format="csc")

In [11]:
def save_step(sol, proj_q, proj_psi, saver, i):
    ins = list()

    ins.append((sd, "cell_q", ( proj_q @ sol[-1][:dof_q]).reshape((3, -1), order="F")))
    ins.append((sd, "cell_p", proj_psi @ sol[-1][dof_q:(dof_q+dof_p)]))
    print( sol[-1][-dof_eta:] )
    
    saver.write_vtu(ins, time_step=i)

In [12]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [13]:
sol = [np.zeros(dof_p + dof_q + dof_eta)]
sol[-1][dof_q:(dof_q+dof_p)] = head_discretization_field.interpolate(sd, initial_h_func)
sol[-1][-dof_eta:] = np.ones_like(sol[-1][-dof_eta:])

proj_q = velocity_discretization_field.eval_at_cell_centers(sd)
proj_psi = head_discretization_field.eval_at_cell_centers(sd)

saver = pp.Exporter(mdg, 'sol', folder_name=output_directory)
save_step(sol, proj_q, proj_psi, saver, 0)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [14]:
for i in range(1, int(T/dt)+1):
    rhs = fixex_rhs.copy()
    rhs[dof_q:(dof_q+dof_p)] += S_s * head_discretization_field.assemble_mass_matrix(sd) @ sol[-1][dof_q:(dof_q+dof_p)]
    rhs[-dof_eta:] += phi * M_gamma @ sol[-1][-dof_eta:]
    
    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc(np.hstack(bc_ess), np.zeros(dof_q + dof_p + dof_eta))
    sol.append( ls.solve() )

    save_step(sol, proj_q, proj_psi, saver, i)

    print('Time ' + str(dt * i) )

saver.write_pvd([t * dt for t in range(int(T/dt)+1)])

[0.86686099 0.86686095 0.86685688 0.86685141 0.86684551 0.86683951
 0.8668336  0.86682814 0.86682409 0.86682407]
Time 0.1
[0.85206594 0.85203731 0.85201899 0.85200522 0.85199347 0.8519823
 0.85197055 0.85195677 0.85193845 0.85190979]
Time 0.2
[0.85857669 0.85855456 0.85853767 0.85852388 0.85851171 0.85850004
 0.85848787 0.85847408 0.85845719 0.85843505]
Time 0.30000000000000004
[0.86903348 0.86901972 0.86900699 0.86899547 0.86898475 0.86897429
 0.86896357 0.86895205 0.86893932 0.86892556]
Time 0.4
[0.87963084 0.87962267 0.87961359 0.87960453 0.87959565 0.87958686
 0.87957799 0.87956893 0.87955984 0.87955168]
Time 0.5
[0.88957477 0.88956992 0.88956354 0.8895566  0.8895495  0.88954236
 0.88953525 0.88952831 0.88952193 0.88951709]
Time 0.6000000000000001
[0.89874373 0.89874082 0.89873636 0.89873113 0.89872556 0.8987199
 0.89871434 0.89870911 0.89870464 0.89870173]
Time 0.7000000000000001
[0.90716244 0.90716068 0.90715755 0.90715366 0.90714938 0.90714498
 0.9071407  0.9071368  0.90713368 0